# Week 3 Assignment
## Multi Armed Bandits

Let's get into some coding for this. 
Multi-Armed Bandits are fun because you get a bunch of interesting and intuitive algorithms for solving a straightforward task: figure out the distribution of the rewards for the arms and thus be able to compare and figure out the best arm.

### Tasks
- Using `epsilon_greedy` as a reference (complete code is provided), write code in `klucb`, `ucb` and `thompson` so that agents following that algorithm can play the game correctly.
- Each agent is derived from the baseclass in the first code block. Please don't make changes to it

### Some Points on Assignment Structure
- Here we have tried to give you a brief insight into how object oriented programming works. This is very useful and allows for programs to be modular and thus reused and replaced as and when necesssary without hassle.
- This shouldn't pose an issue in the logic building, but in case it does, or you have related doubts, feel free to drop us a text.

In [1]:
# Don't make any changes to this

import numpy as np
import random
import matplotlib.pyplot as plt

class MultiArmedBandit:
    def __init__(self, arms : np.ndarray[float]):
        self.arms = arms # list of probabilities of each arm returning a favorable reward
        self.best_arm = np.max(arms) # useful for regret calculation
        self.cumulative_regret_array = [0]

    def pull(self, arm:int) -> int:
        assert arm in np.arange(0, len(self.arms)), "Action undefined for bandit"
        reward = 1 if np.random.random() < self.arms[arm] else 0
        self.cumulative_regret_array.append(self.cumulative_regret_array[-1] + self.best_arm - reward)
        return reward
    
    def plot_cumulative_regret(self):
        timesteps = np.arange(1, len(self.cumulative_regret_array) + 1)

        # Plot the data
        plt.figure(figsize=(8,4))
        plt.plot(timesteps, self.cumulative_regret_array, linestyle='-', color='r', label='Cumulative Regret')

        # Formatting
        plt.title('Cumulative Regret Over Time', fontsize=16)
        plt.xlabel('Timesteps', fontsize=14)
        plt.ylabel('Cumulative Regret', fontsize=14)
        plt.grid(True, which='both', linestyle='-', linewidth=0.5)
        plt.yticks(np.arange(0, max(self.cumulative_regret_array) + 5, step=5))

        # Add legend
        plt.legend(loc='upper left', fontsize=12)

        # Tight layout to ensure there's no clipping of labels
        plt.tight_layout()

        # Show plot
        plt.show()



class Agent:
    def __init__(self, time_to_run, bandit : MultiArmedBandit):
        self.time_to_run = time_to_run
        self.rewards = []
        self.bandit = bandit
        self.arms = len(bandit.arms)
    
    def plot_reward_vs_time_curve(self):

        # Create an index for timesteps
        timesteps = np.arange(1, len(self.rewards) + 1)

        # Average out self.rewards
        avg_rewards = [np.mean(self.rewards[0:T+1]) for T in range(self.time_to_run)]

        # Plot the data
        plt.figure(figsize=(8,4))
        plt.plot(timesteps, avg_rewards, linestyle='-', color='g', label='Rewards')

        # Formatting
        plt.title('Average Reward Over Time', fontsize=16)
        plt.xlabel('Timesteps', fontsize=14)
        plt.ylabel('Mean Reward Value upto timestep t', fontsize=14)
        plt.grid(True, which='both', linestyle='-', linewidth=0.5)
        # plt.xticks(timesteps)  # Show all timesteps as x-axis ticks
        # plt.yticks(np.arange(0, max(self.rewards) + 5, step=5))

        # Add legend
        plt.legend(loc='upper left', fontsize=12)

        # Tight layout to ensure there's no clipping of labels
        plt.tight_layout()

        # Show plot
        plt.show()


## Epsilon Greedy Agent

In [2]:
# This is properly written (just need to run)

class EpsilonGreedyAgent(Agent):
    epsilon : float
    reward_memory : np.ndarray # A per arm value of how much reward was gathered
    count_memory : np.ndarray # An array of the number of times an arm is pulled 

    def __init__(self, time_horizon, bandit:MultiArmedBandit, epsilon = 0.01):
        super().__init__(time_horizon, bandit)
        self.epsilon = epsilon
        self.bandit : MultiArmedBandit = bandit
        self.reward_memory = np.zeros(len(bandit.arms))
        self.count_memory = np.zeros(len(bandit.arms))
        self.time_step = 0

    def give_pull(self):
        if np.random.random() < self.epsilon: # Choose random action
           random_arm = np.random.choice(len(self.bandit.arms))
           reward =  self.bandit.pull(random_arm)
           self.reinforce(reward, random_arm)
           self.count_memory[random_arm] += 1
           self.reward_memory[random_arm] += reward
        else: # Choose best known action!
           best_arm = np.argmax(self.reward_memory / self.count_memory)
           reward = self.bandit.pull(best_arm)
           self.reinforce(reward, best_arm)
           

    def reinforce(self, reward, arm):
        self.count_memory[arm] += 1
        self.reward_memory[arm] += reward
        self.time_step += 1
        self.rewards.append(reward)
        # print(self.count_memory, self.reward_memory)
 
    def plot_arm_graph(self):
        counts = self.count_memory
        indices = np.arange(len(counts))

        # Plot the data
        plt.figure(figsize=(12, 6))
        plt.bar(indices, counts, color='skyblue', edgecolor='black')

        # Formatting
        plt.title('Counts per Category', fontsize=16)
        plt.xlabel('Arm', fontsize=14)
        plt.ylabel('Pull Count', fontsize=14)
        plt.grid(axis='y', linestyle='-')  # Add grid lines for the y-axis
        plt.xticks(indices, [f'Category {i+1}' for i in indices], rotation=45, ha='right')
        # plt.yticks(np.arange(0, max(counts) + 2, step=2))

        # Annotate the bars with the count values
        for i, count in enumerate(counts):
            plt.text(i, count + 0.5, str(count), ha='center', va='bottom', fontsize=12, color='black')

        # Tight layout to ensure there's no clipping of labels
        plt.tight_layout()

        # Show plot
        plt.show()    

In [ ]:
# Code to test
# Init Bandit
TIME_HORIZON = 10_000
bandit = MultiArmedBandit(np.array([0.23,0.55,0.76,0.44]))
agent = EpsilonGreedyAgent(TIME_HORIZON, bandit, 0.05)

# Loop
for i in range(TIME_HORIZON):
    agent.give_pull()

# Plot curves
agent.plot_reward_vs_time_curve()
agent.plot_arm_graph()
bandit.plot_cumulative_regret()


## UCB

In [ ]:
class UCBAgent(Agent):
    # Add fields 

    def __init__(self, time_horizon, bandit:MultiArmedBandit,): 
        # Add fields
        super().__init__(time_horizon, bandit)

    def give_pull(self):
        raise NotImplementedError 

    def reinforce(self, reward, arm):
        raise NotImplementedError
 
    def plot_arm_graph(self):
        raise NotImplementedError

In [ ]:
# Code to test
# Init Bandit
TIME_HORIZON = 10_000
bandit = MultiArmedBandit(np.array([0.23,0.55,0.76,0.44]))
agent = None ## Fill with correct constructor

# Loop
for i in range(TIME_HORIZON):
    agent.give_pull()

# Plot curves
agent.plot_reward_vs_time_curve()
agent.plot_arm_graph()
bandit.plot_cumulative_regret()

## KL-UCB

In [ ]:
class KLUCBAgent(Agent):
    # Add fields 

    def __init__(self, time_horizon, bandit:MultiArmedBandit,): 
        # Add fields
        super().__init__(time_horizon, bandit)

    def give_pull(self):
        raise NotImplementedError 

    def reinforce(self, reward, arm):
        raise NotImplementedError
 
    def plot_arm_graph(self):
        raise NotImplementedError

In [ ]:
# Code to test
# Init Bandit
TIME_HORIZON = 10_000
bandit = MultiArmedBandit(np.array([0.23,0.55,0.76,0.44]))
agent = None ## Fill with correct constructor

# Loop
for i in range(TIME_HORIZON):
    agent.give_pull()

# Plot curves
agent.plot_reward_vs_time_curve()
agent.plot_arm_graph()
bandit.plot_cumulative_regret()

## Thompson Sampling

In [ ]:
class ThompsonSamplingAgent(Agent):
    # Add fields 

    def __init__(self, time_horizon, bandit:MultiArmedBandit,): 
        # Add fields
        super().__init__(time_horizon, bandit)

    def give_pull(self):
        raise NotImplementedError 

    def reinforce(self, reward, arm):
        raise NotImplementedError
 
    def plot_arm_graph(self):
        raise NotImplementedError

In [ ]:
# Code to test
# Init Bandit
TIME_HORIZON = 10_000
bandit = MultiArmedBandit(np.array([0.23,0.55,0.76,0.44]))
agent = None ## Fill with correct constructor

# Loop
for i in range(TIME_HORIZON):
    agent.give_pull()

# Plot curves
agent.plot_reward_vs_time_curve()
agent.plot_arm_graph()
bandit.plot_cumulative_regret()

### What's required in `main` ?
Once all agents are ready, let them all try their hands at the same games (each run should result the same reward sequence for every agent playing the game). 

The first set of games, S1, will be vanilla (again 30k time steps). This set just contains the single bandit game given in the main section of any of those files you saw. Run that instance and plot a single curve (one for cumulative regret, one for cumulative reward) for all the 4 agents. Also plot a bar chart of the number of times each agent chose the optimal arm alone.


The next set of games we'll call S2 include a pair of bandits of probability $p$ and $p+0.1$ where $p \in$ \{0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9\}. I.e., in each game, there'll be two bandits, and there'll be 17 such games. For S2, cumulative reward and regret curves are not required to be plotted. What will be required is a plot of the final regret at the end of the games for each value of $p$. This should be easy. The hard part is to explain why the curves you obtained for KL-UCB and UCB look the way they look. Explain the answer to this riddle in a markdown cell at the end.

In [ ]:
# main
# Write code which will run all the different bandit agents together and:
# 1. Plot a common cumulative regret curves graph
# 2. Plot a common graph of average reward curves

# Solve S2

### Answer to S2